# TransformChain

In [ ]:
import openai
import os 
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
import re

In [ ]:
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
llm = ChatOpenAI()

In [ ]:
with open("nda.txt", 'r', encoding='utf-8') as file:
    nda = file.read()


In [ ]:
print(nda)

In [ ]:
match = re.search(r"Exclusions from Confidential Information:(.*?)(?:\n\n|$)", nda, re.DOTALL)
section = match.group(1).strip()
print(section)

In [ ]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    match = re.search(r"Exclusions from Confidential Information:(.*?)(?:\n\n|$)", text, re.DOTALL)
    section = match.group(1).strip()
    return {"output_text": section}

transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=transform_func
)

In [ ]:
template = """Summarize this text:

{output_text}

Summary:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)

In [ ]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain],verbose=True)

In [ ]:
sequential_chain.run(nda)